# **START SESSION**

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import KMeansModel
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer

import matplotlib.pyplot as plt
import pandas as pd
import tempfile
import numpy as np
import seaborn as sns 
%matplotlib inline

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("Proyek PBD Spark Cluster E-Commerce") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
spark

# **CONVERT AND LOAD DATA**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data = spark.read.load("/content/drive/MyDrive/PROYEK PBD/dataset/data.csv",
                     format="csv", inferSchema="true", header="true")

Konversi format data yang sebelumnya berupa format CSV, menjadi Hadoop Format file dalam format **Parquet**.

In [ ]:
data.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/PROYEK PBD/dataset/data_parq")

In [ ]:
data_parq = spark.read.load("/content/drive/MyDrive/PROYEK PBD/dataset/data_parq",
                     format="parquet", inferSchema="true", header="true")

# **EXPLORATORY DATA**

In [ ]:
data_parq.show(10)
data_parq.printSchema()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [ ]:
#check deskripsi dataset
data_parq.describe().show()

+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|    InvoiceDate|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|            541909|         541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|           null|4.611113626082972|15287.690570239585|       null|
| stddev|13428.417280800133| 16799.73762842775|                null|218.08115785023486|           null| 96.7598530611797| 1713.600303321594|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -8

# **DATA CLEANING AND DATA MANIPULATION**

**CLEANING DATA FROM NULL VALUES**

In [ ]:
from pyspark.sql.functions import count
#check and remove the null values
def my_count(df_in):
    df_in.agg( *[ count(c).alias(c) for c in df_in.columns ] ).show()

In [ ]:
my_count(data_parq)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   541909|   541909|     540455|  541909|     541909|   541909|    406829| 541909|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



Melalui proses penghitungan, ditemukan sejumlah baris data pada setiap kolom tidak sama, pada kolom **CustomerID** terdapat nilai null. Maka record data yang memiliki nilai null akan dihapus dari frame data dengan menggunakan fungsi .dropna()

In [ ]:
ecom_data = data_parq.dropna(how='any')
my_count(ecom_data)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   406829|   406829|     406829|  406829|     406829|   406829|    406829| 406829|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



**DATA MANIPULATION**

**1. Manipulasi Perhitungan Total Price**

Dilakukan perhitungan untuk kolom baru yakni Total Price dangan menggunakan perkalian antara Quantity dengan UnitPrice (Quantity * Unit Price)

In [ ]:
from pyspark.sql.functions import round

ecom_data = ecom_data.withColumn('TotalPrice', round( ecom_data.Quantity * ecom_data.UnitPrice, 2 ) )
ecom_data.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|TotalPrice|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|      15.3|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|      22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     20.34|
+---------+---------+--------------------+--------+--------------+---------+----------+-

**2. Manipulasi Format Tanggal**

Jika dilihat di kolom InvoiceData masih dalam format MM/dd/yy HH:mm. Dikarenakan Spark sangat sensitive terhadap format tanggal, maka diperlukan membuat sebuah kolom baru dengan nama kolom **NewInvoiceDate** dengan menggunakan value yang terdapat pada kolom InvoiceDate

In [ ]:
from pyspark.sql.functions import to_date
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
ecom_data.withColumn("NewInvoiceDate", to_date(col("InvoiceDate"), "MM/dd/yyyy HH:mm")).show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|TotalPrice|NewInvoiceDate|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|      15.3|    2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     20.34|    2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|      22.0|    2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     20.34|    2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kin

**3. Manipulasi Perhitungan Time Difference (Duration)**

In [ ]:
from pyspark.sql.functions import mean, min, max, sum, datediff, to_date

date_max = ecom_data.select(max('NewInvoiceDate')).toPandas()
current = to_utc_timestamp( unix_timestamp(lit(str(date_max.iloc[0][0])), \
                              'yy-MM-dd HH:mm').cast('timestamp'), 'UTC' )

#Calculatre Duration
ecom_data = ecom_data.withColumn('Duration', datediff(lit(current), 'NewInvoiceDate'))


In [ ]:
ecom_data.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-------------------+--------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|TotalPrice|     NewInvoiceDate|Duration|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-------------------+--------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|      15.3|2010-12-01 08:26:00|     373|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     20.34|2010-12-01 08:26:00|     373|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|      22.0|2010-12-01 08:26:00|     373|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     20.34|2010-12-01 08:26:00|     373|

# **RFM ANALYSIS METHODOLOGY**

RFM (Recency, Frequency, and Monetary) merupakan sebuah metode yang digunakan untuk menganalisis value dari customer. RFM biasanya digunakan pada basisdata marketing dan direct marketing dan telah mendapatkan perhatian khusus dalam industri jasa retail dan profesional. Berikut merupakan rincian lebih lanjut mengenai RFM:

*   Recency Value, merepresentasikan seberapa baru pelanggan membeli sesuatu? misalnya: durasi sejak pembelian terakhir
*   Frequency Value, merepresentasikan seberapa sering pelanggan membeli? misalnya: Jumlah total pembelian
*   Monetary Value, merepresentasikan seberapa banyak yang dibelanjakan pelanggan? misalnya: Total uang yang dihabiskan pelanggan

# **RFM SEGMENTATION**

**BUILD THE REGENCY, FREQUENCY AND MONETARY**

In [ ]:
recency = ecom_data.groupBy('CustomerID').agg(min('Duration').alias('Recency'))

frequency = ecom_data.groupBy('CustomerID', 'InvoiceNo').count()\
                        .groupBy('CustomerID')\
                        .agg(count("*").alias("Frequency"))

monetary = ecom_data.groupBy('CustomerID').agg(round(sum('TotalPrice'), 2).alias('Monetary'))

rfm = recency.join(frequency,'CustomerID', how = 'inner')\
             .join(monetary,'CustomerID', how = 'inner')

In [ ]:
rfm.show(10)

+----------+-------+---------+--------+
|CustomerID|Recency|Frequency|Monetary|
+----------+-------+---------+--------+
|     15619|     10|        1|   336.4|
|     17389|      0|       43|31300.08|
|     12940|     46|        4|  876.29|
|     13623|     30|        7|  672.44|
|     14450|    180|        3|  483.25|
|     15727|     16|        7| 5178.96|
|     15790|     10|        1|  220.85|
|     13285|     23|        4| 2709.12|
|     14570|    280|        2|  218.06|
|     16574|     71|        1|  451.44|
+----------+-------+---------+--------+
only showing top 10 rows



**SEGMENTASI RFM**

**A. Menentukan Cutting Point**

*Cutting Point* merupakan sebuah metode yang digunakan untuk memilah penggunaan suatu kriteria untuk dijadikan sebagai pertimbangan dalam pengambilan sebuah keputusan terhadap suatu masalah. Metode *Cutting Point* juga digunakan untuk memastikan seberapa penting derajat sebuah kriteria.

**B. Fungsi Mendefenisikan User dengan menggunakan persen Kuartil sebagai Cutting Points**

Kuartil digunakan untuk membagi populasi kedalam berbagai kelompok. Misalnya untuk menemukan 25% total pengeluaran uang teratas (Monetary) pada populasi data online retail yang digunakan pada proyek adalah sebesar 293.362 

In [ ]:
def describe_pd(df_in, columns, style):
    '''
    Function to union the basic stats results and deciles
    :param df_in: the input dataframe
    :param columns: the cloumn name list of the numerical variable
    :param style: the display style
    :return : the numerical describe info. of the input dataframe
    :author: MIng Chen and Wenqiang Feng
    :email: von198@gmail.com
    '''
    if style == 1:
        percentiles = [25, 50, 75]
    else:
        percentiles = np.array(range(0, 110, 10))
    
    percs = np.transpose([np.percentile(df_in.select(x).collect(), percentiles) for x in columns])
    percs = pd.DataFrame(percs, columns=columns)
    percs['summary'] = [str(p) + '%' for p in percentiles]
    
    spark_describe = df_in.describe().toPandas()
    new_df = pd.concat([spark_describe, percs],ignore_index=True)
    new_df = new_df.round(2)
    return new_df[['summary'] + columns]

In [ ]:
cols = ['Recency','Frequency','Monetary']
describe_pd(rfm,cols,1)

,summary,Recency,Frequency,Monetary
0,count,4372,4372,4372
1,mean,91.58119853613907,5.07548032936871,1898.4597003659658
2,stddev,100.77213931384833,9.338754163574729,8219.345141139722
3,min,0,1,-4287.63
4,max,373,248,279489.02
5,25%,16,1,293.362
6,50%,50,3,648.075
7,75%,143,5,1611.72


**C. Mendefenisikan score segmen RFM berdasarkan quartil**

***Kuartil 1 (Q1) = 25%***

*   Untuk score R (Recency) <= 16
*   Untuk score F (Frequency) <= 1
*   Untuk score M (Monetary) <= -4287.63

***Kuartil 2 (Q2) = 50%***

*   Untuk score R (Recency) <= 50
*   Untuk score F (Frequency) <= 3
*   Untuk score M (Monetary) <= 648.075

***Kuartil 3 (Q3) = 70%***

*   Untuk score R (Recency) <= 143
*   Untuk score F (Frequency) <= 5
*   Untuk score M (Monetary) <= 1611.72

***Kuartil 4 (Q4)***

Score R, F, M yang tepat berada lebih besar dari score untuk Kuartil 3 (70%)

In [ ]:
def RScore(x):
    if  x <= 16:
        return 1
    elif x<= 50:
        return 2
    elif x<= 143:
        return 3
    else:
        return 4

def FScore(x):
    if  x <= 1:
        return 4
    elif x <= 3:
        return 3
    elif x <= 5:
        return 2
    else:
        return 1

def MScore(x):
    if  x <= 293:
        return 4
    elif x <= 648:
        return 3
    elif x <= 1611:
        return 2
    else:
        return 1

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType

R_udf = udf(lambda x: RScore(x), StringType())
F_udf = udf(lambda x: FScore(x), StringType())
M_udf = udf(lambda x: MScore(x), StringType())

**D. Implementasi Segmentasi RFM**

In [ ]:
rfm_seg = rfm.withColumn("r_seg", R_udf("Recency"))
rfm_seg = rfm_seg.withColumn("f_seg", F_udf("Frequency"))
rfm_seg = rfm_seg.withColumn("m_seg", M_udf("Monetary"))
rfm_seg.show(5)

+----------+-------+---------+--------+-----+-----+-----+
|CustomerID|Recency|Frequency|Monetary|r_seg|f_seg|m_seg|
+----------+-------+---------+--------+-----+-----+-----+
|     15619|     10|        1|   336.4|    1|    4|    3|
|     17389|      0|       43|31300.08|    1|    1|    1|
|     12940|     46|        4|  876.29|    2|    2|    2|
|     13623|     30|        7|  672.44|    2|    1|    2|
|     14450|    180|        3|  483.25|    4|    3|    3|
+----------+-------+---------+--------+-----+-----+-----+
only showing top 5 rows



**E. Menentukan score RFM**

Dengan menggabungkan score dari masing-masing segmen R, F, dan M menggunakan fungsi Concat

In [ ]:
from pyspark.sql import functions as F
rfm_seg = rfm_seg.withColumn('RFMScore',
                             F.concat(F.col('r_seg'),F.col('f_seg'), F.col('m_seg')))
rfm_seg.sort(F.col('RFMScore')).show(10)

+----------+-------+---------+--------+-----+-----+-----+--------+
|CustomerID|Recency|Frequency|Monetary|r_seg|f_seg|m_seg|RFMScore|
+----------+-------+---------+--------+-----+-----+-----+--------+
|     13468|      1|       41| 5656.75|    1|    1|    1|     111|
|     15594|     15|        6| 1767.53|    1|    1|    1|     111|
|     15382|     14|        8| 5927.86|    1|    1|    1|     111|
|     17754|      0|        6| 1739.92|    1|    1|    1|     111|
|     16549|     10|       10| 4154.64|    1|    1|    1|     111|
|     17389|      0|       43|31300.08|    1|    1|    1|     111|
|     12471|      2|       49|18740.92|    1|    1|    1|     111|
|     15727|     16|        7| 5178.96|    1|    1|    1|     111|
|     17809|     16|       15| 4627.62|    1|    1|    1|     111|
|     18161|     10|        6| 1612.79|    1|    1|    1|     111|
+----------+-------+---------+--------+-----+-----+-----+--------+
only showing top 10 rows



**F. Summary RFM Segmentasi**

In [ ]:
rfm_seg.groupBy('RFMScore')\
       .agg({'Recency':'mean',
             'Frequency': 'mean',
             'Monetary': 'mean'} )\
        .sort(F.col('RFMScore')).show(10)

+--------+-----------------+------------------+------------------+
|RFMScore|     avg(Recency)|     avg(Monetary)|    avg(Frequency)|
+--------+-----------------+------------------+------------------+
|     111|6.035123966942149| 8828.888595041324|18.882231404958677|
|     112|7.237113402061856|1223.3604123711339| 7.752577319587629|
|     113|              8.0|          505.9775|               7.5|
|     114|             11.0|            191.17|               8.0|
|     121|6.472727272727273|2569.0619999999994| 4.636363636363637|
|     122|7.339622641509434|1022.4575471698115| 4.452830188679245|
|     123|7.935483870967742| 522.3993548387094| 4.129032258064516|
|     124|              9.6|            207.24|               4.4|
|     131|7.533333333333333|3260.5020000000004|2.6666666666666665|
|     132|7.329113924050633| 965.5102531645567|2.5822784810126582|
+--------+-----------------+------------------+------------------+
only showing top 10 rows



# **K-MEANS CLUSTERING**

**1. BUILD DENSE FEATURE MATRIX**

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# Method (good for large features):
def transData(data):
    return data.rdd.map(lambda r: [r[0],Vectors.dense(r[1:])]).toDF(['CustomerID','rfm'])

In [ ]:
transformed = transData(rfm)
transformed.show(5)

+----------+-------------------+
|CustomerID|                rfm|
+----------+-------------------+
|     15619|   [10.0,1.0,336.4]|
|     17389|[0.0,43.0,31300.08]|
|     12940|  [46.0,4.0,876.29]|
|     13623|  [30.0,7.0,672.44]|
|     14450| [180.0,3.0,483.25]|
+----------+-------------------+
only showing top 5 rows



**2. DATA NORMALIZATION**

Melakukan Normalisasi data dengan menggunakan MinMax Scaler yang bekerja men-scaling/menyesuaikan data dalam rentang tertentu (range nilai minimum hingga nilai maksimum)

In [ ]:
#Scaler the feature matrix
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol="rfm",\
         outputCol="features")
scalerModel =  scaler.fit(transformed)
scaledData = scalerModel.transform(transformed)
scaledData.show(5,False)

+----------+-------------------+---------------------------------------------------------------+
|CustomerID|rfm                |features                                                       |
+----------+-------------------+---------------------------------------------------------------+
|15619     |[10.0,1.0,336.4]   |[0.02680965147453083,0.0,0.016294610567853272]                 |
|17389     |[0.0,43.0,31300.08]|[0.0,0.1700404858299595,0.12540746393334334]                   |
|12940     |[46.0,4.0,876.29]  |[0.12332439678284182,0.012145748987854251,0.01819712791732512] |
|13623     |[30.0,7.0,672.44]  |[0.08042895442359249,0.024291497975708502,0.017478781288030567]|
|14450     |[180.0,3.0,483.25] |[0.48257372654155495,0.008097165991902834,0.016812095004997765]|
+----------+-------------------+---------------------------------------------------------------+
only showing top 5 rows



**3. MENENTUKAN JUMLAH CLUSTER OPTIMAL**

Cara yang paling baik digunakan untuk menentukan jumlah cluster yang paling optimal adalah dapat dengan menggunakan analisis Silhouette

**a. Nilai Silhouette apabila diassign K = 3**

In [ ]:
import numpy as np
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import KMeansModel
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt
%matplotlib inline 

kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(scaledData)

#Make predictions, i.e., get cluster index
predictions = model.transform(scaledData)

#Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouettes = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouettes))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.8045154385557955
Cluster Centers: 
[0.08223703 0.02240483 0.02410349]
[0.79062677 0.00226695 0.01653601]
[0.40872322 0.00631499 0.01759697]


**b. Nilai Silhouette apabila diassign K = 4**

In [ ]:
kmeans_4 = KMeans().setK(4).setSeed(1)
model = kmeans_4.fit(scaledData)

#Make predictions, i.e., get cluster index
predictions = model.transform(scaledData)

#Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouettes = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouettes))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.7009137758888742
Cluster Centers: 
[0.51005792 0.00498936 0.0171829 ]
[0.04826729 0.02724511 0.02602839]
[0.82944081 0.00216831 0.01637697]
[0.21414675 0.0083373  0.01853346]


**c. Nilai Silhouette apabila diassign K = 5**

In [ ]:
kmeans_5 = KMeans().setK(5).setSeed(1)
model = kmeans_5.fit(scaledData)

#Make predictions, i.e., get cluster index
predictions = model.transform(scaledData)

#Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouettes = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouettes))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.6713448363173243
Cluster Centers: 
[0.65278766 0.00285512 0.01685058]
[0.04338795 0.02841605 0.02649592]
[0.89527268 0.00219085 0.01625428]
[0.18408955 0.00881478 0.01867881]
[0.41229736 0.00625071 0.01752903]


**4. IMPLEMENTASI K-MEANS CLUSTERING**

In [ ]:
k = 3
kmeans = KMeans().setK(k).setSeed(1)
model = kmeans.fit(scaledData)
#Make predictions
predictions = model.transform(scaledData)
predictions.show(5,False)

+----------+-------------------+---------------------------------------------------------------+----------+
|CustomerID|rfm                |features                                                       |prediction|
+----------+-------------------+---------------------------------------------------------------+----------+
|15619     |[10.0,1.0,336.4]   |[0.02680965147453083,0.0,0.016294610567853272]                 |0         |
|17389     |[0.0,43.0,31300.08]|[0.0,0.1700404858299595,0.12540746393334334]                   |0         |
|12940     |[46.0,4.0,876.29]  |[0.12332439678284182,0.012145748987854251,0.01819712791732512] |0         |
|13623     |[30.0,7.0,672.44]  |[0.08042895442359249,0.024291497975708502,0.017478781288030567]|0         |
|14450     |[180.0,3.0,483.25] |[0.48257372654155495,0.008097165991902834,0.016812095004997765]|2         |
+----------+-------------------+---------------------------------------------------------------+----------+
only showing top 5 rows



**5. STATISTICAL SUMMARY**

In [ ]:
results = rfm.join(predictions.select('CustomerID','prediction'),'CustomerID',how='left')
results.show(10)

+----------+-------+---------+--------+----------+
|CustomerID|Recency|Frequency|Monetary|prediction|
+----------+-------+---------+--------+----------+
|     13098|      1|       41|28658.88|         0|
|     13248|    124|        2|  465.68|         2|
|     13452|    259|        2|   590.0|         1|
|     13460|     29|        2|  183.44|         0|
|     13518|     85|        1|  659.44|         0|
|     13638|     15|        1|  122.64|         0|
|     13723|    217|        1|  199.85|         2|
|     14117|    143|        1|    90.0|         2|
|     14719|      1|        6| 1592.18|         0|
|     15057|    275|        2|  1489.5|         1|
+----------+-------+---------+--------+----------+
only showing top 10 rows



In [ ]:
finalrfm = results.withColumn("r_seg", R_udf("Recency"))
finalrfm = finalrfm.withColumn("f_seg", F_udf("Frequency"))
finalrfm = finalrfm.withColumn("m_seg", M_udf("Monetary"))
finalrfm.show(5)

+----------+-------+---------+--------+----------+-----+-----+-----+
|CustomerID|Recency|Frequency|Monetary|prediction|r_seg|f_seg|m_seg|
+----------+-------+---------+--------+----------+-----+-----+-----+
|     13098|      1|       41|28658.88|         0|    1|    1|    1|
|     13248|    124|        2|  465.68|         2|    3|    3|    3|
|     13452|    259|        2|   590.0|         1|    4|    3|    3|
|     13460|     29|        2|  183.44|         0|    2|    3|    4|
|     13518|     85|        1|  659.44|         0|    3|    4|    2|
+----------+-------+---------+--------+----------+-----+-----+-----+
only showing top 5 rows



In [ ]:
datafinal = finalrfm.withColumn('RFMScore',
                             F.concat(F.col('r_seg'),F.col('f_seg'), F.col('m_seg')))
datafinal.show(10)

+----------+-------+---------+--------+----------+-----+-----+-----+--------+
|CustomerID|Recency|Frequency|Monetary|prediction|r_seg|f_seg|m_seg|RFMScore|
+----------+-------+---------+--------+----------+-----+-----+-----+--------+
|     13098|      1|       41|28658.88|         0|    1|    1|    1|     111|
|     13248|    124|        2|  465.68|         2|    3|    3|    3|     333|
|     13452|    259|        2|   590.0|         1|    4|    3|    3|     433|
|     13460|     29|        2|  183.44|         0|    2|    3|    4|     234|
|     13518|     85|        1|  659.44|         0|    3|    4|    2|     342|
|     13638|     15|        1|  122.64|         0|    1|    4|    4|     144|
|     13723|    217|        1|  199.85|         2|    4|    4|    4|     444|
|     14117|    143|        1|    90.0|         2|    3|    4|    4|     344|
|     14719|      1|        6| 1592.18|         0|    1|    1|    2|     112|
|     15057|    275|        2|  1489.5|         1|    4|    3|  

In [ ]:
datafinal.toPandas().to_csv("/content/drive/MyDrive/PROYEK PBD/dataset/rfmquantil.csv")

In [ ]:
datafinal.toPandas().to_csv("D:\KULIAH Tingkat 4\rfmquantil.csv")